In [1]:
import numpy as np

from src.health import HealthAnalysis
from src.viz import (hist_bp,
                     box_weight_by_sex,
                     bar_smoker_share,
                     scatter_bp_vs_age,
                     bar_mean_bp_smoker)
np.random.seed(42)

In [4]:
analysis = HealthAnalysis()
df = analysis.df
df.head()
desc = analysis.descriptive()
desc

,mean,median,min,max
metric,,,,
age,49.426250,50.00,18.0,90.00
height,171.849250,171.35,144.4,200.40
weight,73.413000,73.20,33.7,114.40
systolic_bp,149.178625,149.40,106.8,185.90
cholesterol,4.929150,4.97,2.5,7.88
